In [ ]:
from pipeline import *
import pandas as pd

In [ ]:
year = 10

df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players = load_data()

df_teams_merged = global_merge(df_teams, df_teams_post, df_series_post,
                                   df_players, df_players_teams, df_coaches, df_awards_players, year)

df_teams_merged.head(20)